In [ ]:
# Install dependencies
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 798.8 kB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-kxubz5e1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-kxubz5e1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import torch
import clip
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import os
import numpy as py

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load model  and preprocess
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:08<00:00, 41.5MiB/s]


In [ ]:
# Load dataset
dataset = pd.read_csv('/content/drive/MyDrive/Project2_4/dataset.csv')

In [ ]:
# Function for scores with clip
def get_label_scores(image_path, labels):
  image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
  text_inputs = clip.tokenize(labels).to(device)

  with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_inputs)

  image_features = image_features / image_features.norm(dim=1, keepdim = True)
  text_features = text_features / text_features.norm(dim=-1, keepdim=True)
  similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

  scores = {labels[i]: similarity[0][i].item() for i in range(len(labels))}
  return scores

results = []

for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
  image_path = row['image']
  labels = [row['label1'], row['label2'], row['label3']]

  # Get scores
  scores = get_label_scores(image_path, labels)

  # Rank labels based on scores (1 = best, 3 = worst)
  ranked_labels = sorted(scores.items(), key=lambda item: item[1], reverse=True)
  ranked_dict = {ranked_labels[i][0]: i + 1 for i in range(len(ranked_labels))}

    # Add to results
  results.append({
        'image': image_path,
        'label1': labels[0],
        'label2': labels[1],
        'label3': labels[2],
        'label1_rank': ranked_dict[labels[0]],
        'label2_rank': ranked_dict[labels[1]],
        'label3_rank': ranked_dict[labels[2]]
  })

# Convert results to DataFrame
results_df = pd.DataFrame(results)


  0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
results_df.to_csv('/content/drive/MyDrive/Project2_4/CLIP_labels.csv', index=False)